# Transform JSON data into CSV
1. Download raw JSON data from GCS bucket
2. Transform the data
3. Export the data as CSV

In [50]:
from google.cloud import storage

# 1. Set up your Google Cloud Storage client
storage_client = storage.Client()

# 2. Specify your bucket name and file path
bucket_name = 'leverageai-sandbox-data'  # Replace with your actual bucket name
file_path = 'raw/source_name:fpl_api_event/event_id:1/source_date:2024-07-13/data001.json'  # Replace with the path to your file in the bucket

# 3. Download the file
bucket = storage_client.bucket(bucket_name)
blob = bucket.blob(file_path)
blob.download_to_filename('./.data/data001.json')  # Save the file locally

# Get blob metadata
blob.reload()
metadata = blob.metadata

print(f"File downloaded from GCS: {file_path}")
print(f"File metadata: {metadata}")


File downloaded from GCS: raw/source_name:fpl_api_event/event_id:1/source_date:2024-07-13/data001.json
File metadata: {'source_api_version': '', 'source_url': 'https://fantasy.premierleague.com/api/event/1/live', 'source_parameters': '{"event_id": 1}', 'response_headers': '{"Connection": "keep-alive", "Content-Length": "12605", "Server": "nginx", "Content-Type": "application/json", "Allow": "GET, HEAD, OPTIONS", "X-Frame-Options": "DENY", "X-Content-Type-Options": "nosniff, nosniff", "Referrer-Policy": "same-origin, same-origin", "Cross-Origin-Opener-Policy": "same-origin, same-origin", "Cache-Control": "max-age=0, no-cache, no-store, must-revalidate", "Edge-Control": "max-age=86400", "Cross-Origin-Resource-Policy": "same-origin", "Permissions-Policy": "geolocation=(),midi=(),sync-xhr=(),microphone=(),camera=(),magnetometer=(),gyroscope=(),fullscreen=(self),payment=()", "Strict-Transport-Security": "max-age=31536000", "Content-Encoding": "gzip", "Via": "1.1 google, 1.1 varnish, 1.1 var

In [51]:
import json
import pandas as pd

json_filepath = '/home/jon/workbench/github/leverageai/fpl/cloud_functions/process_fpl_event/scratch/.data/data001.json'

# Load json file
with open(json_filepath) as f:
    data = json.load(f)

# Extract elements from raw data into a Pandas DataFrame
elements_df = pd.DataFrame(data['elements'])

# Print the DataFrame
elements_df.head()


,id,stats,explain
0,1,"{'minutes': 0, 'goals_scored': 0, 'assists': 0...","[{'fixture': 2, 'stats': [{'identifier': 'minu..."
1,2,"{'minutes': 0, 'goals_scored': 0, 'assists': 0...","[{'fixture': 2, 'stats': [{'identifier': 'minu..."
2,3,"{'minutes': 0, 'goals_scored': 0, 'assists': 0...","[{'fixture': 2, 'stats': [{'identifier': 'minu..."
3,4,"{'minutes': 0, 'goals_scored': 0, 'assists': 0...","[{'fixture': 2, 'stats': [{'identifier': 'minu..."
4,5,"{'minutes': 4, 'goals_scored': 0, 'assists': 0...","[{'fixture': 2, 'stats': [{'identifier': 'minu..."


In [59]:
elements_stats_df = pd.concat(
    objs=[
        elements_df['id'],
        pd.json_normalize(elements_df['stats'])],
    axis=1
    )
elements_stats_df['event_id'] = json.loads(metadata["source_parameters"])["event_id"]
elements_stats_df

,id,minutes,goals_scored,assists,clean_sheets,goals_conceded,own_goals,penalties_saved,penalties_missed,yellow_cards,...,threat,ict_index,starts,expected_goals,expected_assists,expected_goal_involvements,expected_goals_conceded,total_points,in_dreamteam,event_id
0,1,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0,0.00,0.00,0.00,0.00,0,False,1
1,2,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0,0.00,0.00,0.00,0.00,0,False,1
2,3,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0,0.00,0.00,0.00,0.00,0,False,1
3,4,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0,0.00,0.00,0.00,0.00,0,False,1
4,5,4,0,0,0,0,0,0,0,0,...,0.0,0.0,0,0.00,0.00,0.00,0.02,1,False,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
653,655,3,0,0,0,0,0,0,0,0,...,2.0,0.3,0,0.00,0.01,0.01,0.00,1,False,1
654,656,3,0,0,0,0,0,0,0,0,...,0.0,0.0,0,0.00,0.00,0.00,0.00,1,False,1
655,657,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0,0.00,0.00,0.00,0.00,0,False,1
656,658,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0,0.00,0.00,0.00,0.00,0,False,1


In [60]:
elements_stats_cols = ['id', 'event_id', 'minutes', 'goals_scored', 'assists', 'clean_sheets',
       'goals_conceded', 'own_goals', 'penalties_saved', 'penalties_missed',
       'yellow_cards', 'red_cards', 'saves', 'bonus', 'bps', 'influence',
       'creativity', 'threat', 'ict_index', 'starts', 'expected_goals',
       'expected_assists', 'expected_goal_involvements',
       'expected_goals_conceded', 'total_points', 'in_dreamteam']
elements_stats_df.to_csv(
    ".data/elements_stats.csv",
    columns=elements_stats_cols, 
    index=False)

In [121]:
elements_explain_df = pd.json_normalize(data,'elements')

# df = df.set_index('id')

# explode explain column into rows
elements_explain_df = elements_explain_df.apply(lambda x: x.explode())

# normalise explain column
# pd.json_normalize(pd.json_normalize(df['explain']))

elements_explain_df = pd.concat(
    [
        elements_explain_df['id'],
        pd.json_normalize(elements_explain_df['explain'])
    ],
    axis=1
)
elements_explain_df = elements_explain_df.apply(lambda x: x.explode()).reset_index(drop=True)
elements_explain_df = pd.concat(
    [
        elements_explain_df.loc[:,['id','fixture']],
        pd.json_normalize(elements_explain_df['stats'])
    ], axis=1
)
elements_explain_df

,id,fixture,identifier,points,value
0,1,2,minutes,0,0
1,2,2,minutes,0,0
2,3,2,minutes,0,0
3,4,2,minutes,0,0
4,5,2,minutes,1,4
...,...,...,...,...,...
867,655,7,minutes,1,3
868,656,7,minutes,1,3
869,657,7,minutes,0,0
870,658,7,minutes,0,0


In [124]:
elements_explain_df.columns

Index(['id', 'fixture', 'identifier', 'points', 'value'], dtype='object')

In [125]:
elements_explain_cols = ['id', 'fixture', 'identifier', 'points', 'value']
elements_explain_df.to_csv(
    ".data/elements_explain.csv",
    columns=elements_explain_cols, 
    index=False)